In [1]:
import logging
import telebot
import requests

ModuleNotFoundError: No module named 'telebot'

In [ ]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Telegram Bot Token
bot_token = ''
bot = telebot.TeleBot(bot_token)

# Open Exchange Rates API
openweathermap_api_key = '845924f7efd55d54e86e2f0cb361cfbb'
openweathermap_api_endpoint = 'http://api.openweathermap.org/data/2.5/weather'

# CoinGecko API
currencylayer_api_key = 'da4eb1065d6fc9000eff650dccb68740'
currencylayer_api_endpoint = 'http://api.currencylayer.com/live'
currencylayer_api_endpoint_convert = 'http://api.currencylayer.com/convert'
valid_currencies = ['rub', 'usd', 'eur', 'cny']

user_data_dict = {}

asked_name = False

In [ ]:
class CurrencyConverter:
    def __init__(self, api_key):
        self.api_key = api_key

    def get_exchange_rates(self, from_currency, to_currency):
        params = {'access_key': self.api_key, 'source': from_currency}
        response = requests.get(currencylayer_api_endpoint, params=params)
        if response.status_code == 200:
            return response.json()['quotes'][f'{from_currency}{to_currency}']
        else:
            raise Exception(f"Failed to fetch exchange rates: {response.text}")

    def convert_currency(self, amount, from_currency, to_currency):
        params = {'access_key': self.api_key, 'from': from_currency, 'to': to_currency, 'amount': amount}
        response = requests.get(currencylayer_api_endpoint_convert, params=params)
        return response.json()['result']

# Currency Converter
currency_converter = CurrencyConverter(currencylayer_api_key)

In [ ]:
def get_weather_info(city):
    params = {
        'q': city,
        'appid': openweathermap_api_key,
        'units': 'metric'  # You can change units to 'imperial' for Fahrenheit
    }
    response = requests.get(openweathermap_api_endpoint, params=params)
    if response.status_code == 200:
        weather_data = response.json()
        temperature = weather_data['main']['temp']
        description = weather_data['weather'][0]['description']
        return f'Temperature: {temperature}°C, Description: {description}'
    else:
        raise Exception(f"Failed to fetch weather information: {response.text}")

@bot.message_handler(commands=['setname'])
def set_name(message):
    user_input = message.text.split(maxsplit=1)
    if len(user_input) == 2:
        user_id = message.from_user.id
        user_data_dict[user_id]['name'] = user_input[1]
        bot.send_message(message.chat.id, f"Nice to meet you, {user_input[1]}!")

In [ ]:
def process_request(message):
    user_input = message.text.lower()

    user_id = message.from_user.id

    # Check if the user's data is already stored
    if user_id not in user_data_dict:
        user_data_dict[user_id] = {'name': None}
        bot.send_message(message.chat.id, "Hello! What's your name?")
        return

    if user_data_dict[user_id]['name'] == None:
        user_data_dict[user_id]['name'] = user_input
        bot.send_message(message.chat.id, f"Nice to meet you, {user_input.upper()}!")

    if "convert" in user_input:
        try:
            words = user_input.split()
            from_currency_empty = True
            to_currency_empty = True
            for word in words:
              if word.isdigit():
                amount = float(word)
              if word in valid_currencies and from_currency_empty:
                from_currency = word
                from_currency_empty = False
              elif word in valid_currencies and to_currency_empty:
                to_currency = word
                to_currency_empty = False
        except ValueError:
            bot.send_message(message.chat.id, "Invalid input format. Please use: /convert <amount> <from_currency> to <to_currency>")
            return

        # if the specified currencies are valid
        if from_currency not in valid_currencies or to_currency not in valid_currencies:
            bot.send_message(message.chat.id, "Invalid currency. Please choose from RUB, USD, EUR, or CNY.")
            return

        try:
            # currency conversion
            converted_amount = currency_converter.convert_currency(amount, from_currency.upper(), to_currency.upper())
            bot.send_message(message.chat.id, f"{amount} {from_currency.upper()} is equal to {converted_amount:.2f} {to_currency.upper()}")
        except Exception as e:
            bot.send_message(message.chat.id, f"Error: {str(e)}")

    elif "weather" in user_input:
        try:
            city = user_input.split()[-1]
        except ValueError:
            bot.send_message(message.chat.id, "Invalid input format. Please use: /weather <city>")
            return


        # weather information
        try:
            weather_info = get_weather_info(city)
            bot.send_message(message.chat.id, f"Weather in {city}: {weather_info}")
        except Exception as e:
            bot.send_message(message.chat.id, f"Error: {str(e)}")

    elif "price" in user_input:
        # currency from the user input
        try:
            from_currency_empty = True
            to_currency_empty = True
            words = user_input.split()
            for word in words:
              if word in valid_currencies and from_currency_empty:
                from_currency = word
                from_currency_empty = False
              elif word in valid_currencies and to_currency_empty:
                to_currency = word
                to_currency_empty = False
        except ValueError:
            bot.send_message(message.chat.id, "Invalid input format. Please use: /price <currency>")
            return

        # if the specified currency/cryptocurrency is valid
        if from_currency not in valid_currencies or to_currency not in valid_currencies:
            bot.send_message(message.chat.id, "Invalid currency. Please choose from RUB, USD, EUR, CNY.")
            return

        # current price information
        try:
            price_info = currency_converter.get_exchange_rates(from_currency.upper(), to_currency.upper())
            bot.send_message(message.chat.id, f"Current price of {to_currency}: {price_info} in {from_currency}")
        except Exception as e:
            bot.send_message(message.chat.id, f"Error: {str(e)}")


@bot.message_handler(func=lambda message: True)
def handle_message(message):
    process_request(message)


if __name__ == '__main__':
    bot.polling(none_stop=True)